## 分類用データセット作成Notebook
MovieLensには正例（実際にCVしたレコード）しか含まれていないためネガティブサンプルを作成する。   
またTrain/Eval/Testで分割するときにtimestampでソートしてから適当な数で分割することとしている（今回はTrain:Eval:Test=7:2:1ぐらいの比率で分割）

In [2]:
import random
from time import time
import numpy as np
import pandas as pd

元データの読み込み

In [ ]:
rec_dataset = pd.read_csv('../data/org_20m/ratings.csv', names=['user','item','rating','timestamp'], skiprows=1)
#rec_dataset = pd.read_csv('../data/u.data', sep='\t',
                         #header=None,
                         #names=['user','item','rating','timestamp'])
rec_dataset.head()

In [ ]:
print('Dataset size: {}'.format(len(rec_dataset)))

以下のデータ数でサンプルを作成する（以下はそれぞれの数字／実際はこれの倍の大きさ）   

- MovieLens20M
    - Train: 14,000,000    
    - Eval: 4,000,000    
    - Test: 2,000,263
- MovieLens100k
    - Train: 70,000
    - Eval: 20,000
    - TestL 10,000

### ユーザ・アイテムの組み合わせを適当に作成する関数   
ユーザ・アイテムの辞書を作成しておき、適当に作成された組み合わせがその中に存在していなければリストに加えるというもの。   
ここで作成したリストはあとでDataFrameに変換する

In [ ]:
def create_random_combinations(df):
    pos_set = set(df.iloc[:,:2].itertuples(index=False, name=None))
    user_list = df.user.unique()
    item_list = df.item.unique()
    neg_user = []
    neg_item = []
    
    while len(neg_user) < len(df):
        new_user = random.choice(user_list)
        new_item = random.choice(item_list)
        new_sample = (new_user, new_item)
        if new_sample not in pos_set:
            neg_user.append(new_user)
            neg_item.append(new_item)
    return neg_user, neg_item

In [ ]:
t1 = time()
neg_user, neg_item = create_random_combinations(rec_dataset)
t2 = time()
print('{:.4f} seconds elapsed'.format(t2-t1))

In [ ]:
neg_df = pd.DataFrame(np.concatenate([np.array(neg_user).reshape(-1,1),
                         np.array(neg_item).reshape(-1,1),
                         np.zeros(len(neg_user)).reshape(-1,1)
                        ], axis=1),
                      columns=['user','item','rating'])

neg_df.head()

In [ ]:
# データ型を整える
neg_df.iloc[:,0] = neg_df.iloc[:,0].map(int).map(str)
neg_df.iloc[:,1] = neg_df.iloc[:,1].map(int).map(str)
neg_df.iloc[:,2] = neg_df.iloc[:,2].map(int)
neg_df.head()


In [ ]:
# 元のデータをpositiveデータとするためratingを1.0に変更しdf化

pos_df = rec_dataset.sort_values('timestamp').iloc[:,:3]
del rec_dataset

# データ型を整える
pos_df.iloc[:,0] = pos_df.iloc[:,0].map(int).map(str)
pos_df.iloc[:,1] = pos_df.iloc[:,1].map(int).map(str)
pos_df.iloc[:,2] = pos_df.iloc[:,2].map(int)
pos_df.loc[:,'rating'] = 1

pos_df.head()


In [ ]:
# Train用のデータセット作成
train_file_name = '../data/MovieLens20M/classification/train20m.csv'
pos_df_train = pos_df.iloc[:14000000,:]
neg_df_train = neg_df.iloc[:14000000,:]

# 特に意味はないが固まっているのが気になるためランダムソートしている
dataset_train = pd.concat([pos_df_train, neg_df_train], axis=0).sample(frac=1).reset_index(drop=True)
dataset_train.to_csv(train_file_name, index=None)


In [ ]:
# Eval用のデータセット作成
eval_file_name = '../data/MovieLens20M/classification/eval20m.csv'
pos_df_eval = pos_df.iloc[14000000:18000000,:]
neg_df_eval = neg_df.iloc[14000000:18000000,:]

dataset_eval = pd.concat([pos_df_eval, neg_df_eval], axis=0).sample(frac=1).reset_index(drop=True)
dataset_eval.to_csv(eval_file_name, index=None)


In [ ]:
# Test用のデータセット作成
test_file_name = '../data/MovieLens20M/classification/test20m.csv'
pos_df_test = pos_df.iloc[18000000:,:]
neg_df_test = neg_df.iloc[18000000:,:]

dataset_test = pd.concat([pos_df_test, neg_df_test], axis=0).sample(frac=1).reset_index(drop=True)
dataset_test.to_csv(test_file_name, index=None)


In [ ]:
# 中身の確認
filename = test_file_name
check_df = pd.read_csv(filename)
print('Dataset size: {}'.format(len(check_df)))
print(check_df.dtypes)
check_df.head()

In [3]:
# 目的変数がstrでなければならない場合があるため一旦別ファイルを作成
# 入っているレコードは同じ
train_file_name = '../data/MovieLens20M/classification/train20m.csv'
train_file_name_str = '../data/MovieLens20M/classification_str/train20m.csv'
tmp_df = pd.read_csv(train_file_name)
tmp_df.iloc[:,-1] = tmp_df.iloc[:,-1].map(str)
tmp_df.to_csv(train_file_name_str, index=None)

In [4]:
# Eval
eval_file_name = '../data/MovieLens20M/classification/eval20m.csv'
eval_file_name_str = '../data/MovieLens20M/classification_str/eval20m.csv'
tmp_df = pd.read_csv(eval_file_name)
tmp_df.iloc[:,-1] = tmp_df.iloc[:,-1].map(str)
tmp_df.to_csv(eval_file_name_str, index=None)

In [5]:
# Test
test_file_name = '../data/MovieLens20M/classification/test20m.csv'
test_file_name_str = '../data/MovieLens20M/classification_str/test20m.csv'
tmp_df = pd.read_csv(test_file_name)
tmp_df.iloc[:,-1] = tmp_df.iloc[:,-1].map(str)
tmp_df.to_csv(test_file_name_str, index=None)